In [ ]:
using Plots
using FFTW

const L = 100
const N = 1000
const M = 1000
const dx = L/N
const k0 = 2π/L
const dt = 1e-3

const x = LinRange(0, L - dx, N)
const k = 2*π / L .* collect(1:(N÷2)+1)
const k² = k.*k

F = plan_rfft(x)
B = plan_irfft(k*(1 + 0*im), N)

const V = cos.(k0 .* x)
const FV = F*V
const v = 4 * B*(-im.*k.*FV)

function antialiasing!( F )
    F[(end-(N÷2)÷3)+1:end] .= 0.
end

function euler!(Fdφ, Fφ)
    Fdφ .= -k².*Fφ
    Fdφ .+= (-im.*k) .* (F*(v.*(B*Fφ)))
    Fφ .+= Fdφ.*dt
    antialiasing!(Fφ)
end


In [ ]:
dφ = zeros(N)
a = 2
φ = 5 * exp.( - ((x .- (L/4)).^2) ./ (2*a^2) ) ./ (√(2π*a^2)) 
plot(x, φ)
plot!(x, V)

In [ ]:
plot(x, v)


In [ ]:
Fφ = F*φ
Fdφ = F*dφ
plot(k, real(Fφ))
plot!(k, imag(Fφ))

In [ ]:
antialiasing!(Fφ)
plot(k, real(Fφ))
plot!(k, imag(Fφ))

In [ ]:
φ = B*Fφ
dφ = B*Fdφ
plot(x, φ)

In [ ]:
for t in 1:10_000
    euler!(Fdφ, Fφ)
end

φ = B*Fφ
dφ = B*Fdφ
plot(x, φ, ylims=(-1,1))
plot!(x, V)
 